# PYDATA SLO Meetup November 14, 2018
## Intro to Regularization

In [166]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from scipy.linalg import hilbert
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [154]:
#Set up the problem
#X = np.matrix('.16, .10; .17 .11; 2.02 1.29')
#X = np.matrix('1 3 11 0  -11 -15;18 55 209 15 -198 -277; -23 -33 144 532 259 82;9 55 405 437 -100 -285;3 -4 -111 -180 39 219;-13 -9  202 346  401  253')
X = hilbert(100)
X

array([[1.        , 0.5       , 0.33333333, ..., 0.01020408, 0.01010101,
        0.01      ],
       [0.5       , 0.33333333, 0.25      , ..., 0.01010101, 0.01      ,
        0.00990099],
       [0.33333333, 0.25      , 0.2       , ..., 0.01      , 0.00990099,
        0.00980392],
       ...,
       [0.01020408, 0.01010101, 0.01      , ..., 0.00512821, 0.00510204,
        0.00507614],
       [0.01010101, 0.01      , 0.00990099, ..., 0.00510204, 0.00507614,
        0.00505051],
       [0.01      , 0.00990099, 0.00980392, ..., 0.00507614, 0.00505051,
        0.00502513]])

In [155]:
#S is ill-conditioned, condition = 1 is not singular
np.linalg.cond(X)


2.525412746419138e+19

In [156]:
#Know the true feature weights or covariate estimates
beta = np.matrix(np.ones(X.shape[1])).transpose()
beta[0:4]

matrix([[1.],
        [1.],
        [1.],
        [1.]])

In [157]:
#True y
y = np.matrix(X)*np.matrix(beta)

In [158]:
#Now add noise to y of size ro
#Dimension of X
r = np.matrix(X.shape)[0, 0]
e = np.random.normal(0, .1, r) #compute vector of normal variants mean=0,std=0.1

#Note that nb is 3 by 1, so we need to flip it
ye = y + np.matrix(e).transpose() #add noise
ye[0:4]

matrix([[5.30703933],
        [4.1225196 ],
        [3.62340987],
        [3.41923332]])

### Inference: Try to recover beta

##### OlS, Naive solution 

In [202]:
#using pseudo inverse
beta_ols = np.linalg.pinv(X)*ye
rmse = np.sqrt(mean_squared_error(beta_ols, beta))
print("Root Mean Squared Error: {}".format(rmse))

Root Mean Squared Error: 1292884866842.632


In [203]:
#using sklearn
beta_ols = LinearRegression().fit(X, ye).coef_
rmse = np.sqrt(mean_squared_error(beta_ols.transpose(), beta))
print("Root Mean Squared Error: {}".format(rmse))

Root Mean Squared Error: 4934567707226.843


##### With Regularization

In [207]:
#using L2 norm, lamba is 0.1
beta_ridge = Ridge(alpha = 0.1).fit(X, ye).coef_
rmse = np.sqrt(mean_squared_error(beta_ridge.transpose(), beta))
print("Root Mean Squared Error: {}".format(rmse))

Root Mean Squared Error: 0.6581925519165268


In [208]:
#using L1 norm, lamba is 0.1
beta_lasso = Lasso(alpha = 0.1).fit(X, ye).coef_
rmse = np.sqrt(mean_squared_error(beta_lasso.transpose(), beta))
print("Root Mean Squared Error: {}".format(rmse))

Root Mean Squared Error: 1.0


In [210]:
#Elastic Net uses combination of the two
#For l1_ratio = 0 the penalty is an L2 penalty. 
#For l1_ratio = 1 it is an L1 penalty. 
#For 0 < l1_ratio < 1, the penalty is a combination of L1 and L2.

beta_el = ElasticNet(alpha = 0.1, l1_ratio=0.5).fit(X, ye).coef_
rmse = np.sqrt(mean_squared_error(beta_el.transpose(), beta))
print("Root Mean Squared Error: {}".format(rmse))

Root Mean Squared Error: 0.9964147808523344
